In [1]:
#pip install -q earthaccess

In [24]:
import geopandas as gpd
import earthaccess
import pandas as pd
from osgeo import gdal
import rasterio as rio
import xarray as xr
import rioxarray as rxr

In [8]:
# EarthAccess setup for HLS
# https://github.com/nasa/HLS-Data-Resources/blob/main/python/tutorials/HLS_Tutorial.ipynb

earthaccess.login(persist=True)

# Load sampling locations
I want to find HLS tiles which overlap with these locations. 

In [9]:
# load sampling locations
sampling_points = gpd.read_file('data/nov_3s.geojson')
# only include site 1 for each location
sampling_points = sampling_points.loc[sampling_points.name.str.contains('1')]
bbox = tuple(list(sampling_points.total_bounds))
bbox

(105.942767, 13.525966, 106.0521, 13.575205)

# Query earthaccess for some HLS data between those dates

In [10]:
# look for data over the sampling dates 
# (Nov 20, 21, 29, & 30, 2023)
temporal = ("2023-11-20T00:00:00", "2023-11-30T23:59:59")

In [11]:
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    bounding_box=bbox,
    temporal=temporal,
    count=100
)

Granules found: 23


There are 23(!) images over that short time period. 
I think that's mainly because I have this awkward
situation where my sampling points are right at the corner
of four different sentinel tiles and three different lansat tiles. 
Further complicated by the fact that landsat is resampled to match sentinel tiles. 

In [12]:

import pandas as pd

results_df = pd.json_normalize(results)
results_df.head(5)

size meta.concept-type      meta.concept-id  meta.revision-id  \
0   40.804092           granule  G2805825463-LPCLOUD                 1   
1  171.964798           granule  G2805826666-LPCLOUD                 1   
2    9.418929           granule  G2805817020-LPCLOUD                 1   
3  165.176474           granule  G2805818626-LPCLOUD                 1   
4  210.919179           granule  G2805838792-LPCLOUD                 1   

                       meta.native-id meta.provider-id  \
0  HLS.L30.T48PXA.2023325T031928.v2.0          LPCLOUD   
1  HLS.L30.T48PWA.2023325T031928.v2.0          LPCLOUD   
2  HLS.L30.T48PXV.2023325T031952.v2.0          LPCLOUD   
3  HLS.L30.T48PWV.2023325T031952.v2.0          LPCLOUD   
4  HLS.S30.T48PXA.2023325T032029.v2.0          LPCLOUD   

              meta.format        meta.revision-date  \
0  application/echo10+xml  2023-11-23T06:53:41.934Z   
1  application/echo10+xml  2023-11-23T06:56:55.887Z   
2  application/echo10+xml  2023-11-23T06:38:46.616Z   
3  application/echo10+xml  2023-11-23T06:40:55.807Z   
4  application/echo10+xml  2023-11-23T07:16:54.521Z   

  umm.TemporalExtent.RangeDateTime.BeginningDateTime  \
0                           2023-11-21T03:19:28.281Z   
1                           2023-11-21T03:19:28.281Z   
2                           2023-11-21T03:19:52.180Z   
3                           2023-11-21T03:19:52.180Z   
4                           2023-11-21T03:34:10.537Z   

  umm.TemporalExtent.RangeDateTime.EndingDateTime  ...  \
0                        2023-11-21T03:19:52.180Z  ...   
1                        2023-11-21T03:19:52.180Z  ...   
2                        2023-11-21T03:19:52.180Z  ...   
3                        2023-11-21T03:19:52.180Z  ...   
4                        2023-11-21T03:34:10.537Z  ...   

                  umm.CollectionReference.EntryTitle  \
0  HLS Landsat Operational Land Imager Surface Re...   
1  HLS Landsat Operational Land Imager Surface Re...   
2  HLS Landsat Operational Land Imager Surface Re...   
3  HLS Landsat Operational Land Imager Surface Re...   
4  HLS Sentinel-2 Multi-spectral Instrument Surfa...   

                                     umm.RelatedUrls  \
0  [{'URL': 'https://data.lpdaac.earthdatacloud.n...   
1  [{'URL': 'https://data.lpdaac.earthdatacloud.n...   
2  [{'URL': 'https://data.lpdaac.earthdatacloud.n...   
3  [{'URL': 'https://data.lpdaac.earthdatacloud.n...   
4  [{'URL': 'https://data.lpdaac.earthdatacloud.n...   

  umm.DataGranule.DayNightFlag  \
0                          Day   
1                          Day   
2                          Day   
3                          Day   
4                          Day   

                         umm.DataGranule.Identifiers  \
0  [{'Identifier': 'HLS.L30.T48PXA.2023325T031928...   
1  [{'Identifier': 'HLS.L30.T48PWA.2023325T031928...   
2  [{'Identifier': 'HLS.L30.T48PXV.2023325T031952...   
3  [{'Identifier': 'HLS.L30.T48PWV.2023325T031952...   
4  [{'Identifier': 'HLS.S30.T48PXA.2023325T032029...   

  umm.DataGranule.ProductionDateTime  \
0           2023-11-23T06:51:42.000Z   
1           2023-11-23T06:53:13.000Z   
2           2023-11-23T06:36:47.000Z   
3           2023-11-23T06:36:50.000Z   
4           2023-11-23T07:14:18.000Z   

   umm.DataGranule.ArchiveAndDistributionInformation  \
0  [{'Name': 'Not provided', 'SizeInBytes': 42786...   
1  [{'Name': 'Not provided', 'SizeInBytes': 18031...   
2  [{'Name': 'Not provided', 'SizeInBytes': 98764...   
3  [{'Name': 'Not provided', 'SizeInBytes': 17320...   
4  [{'Name': 'Not provided', 'SizeInBytes': 22116...   

                                       umm.Platforms  \
0  [{'ShortName': 'LANDSAT-8', 'Instruments': [{'...   
1  [{'ShortName': 'LANDSAT-8', 'Instruments': [{'...   
2  [{'ShortName': 'LANDSAT-8', 'Instruments': [{'...   
3  [{'ShortName': 'LANDSAT-8', 'Instruments': [{'...   
4  [{'ShortName': 'Sentinel-2B', 'Instruments': [...   

                       umm.MetadataSpecification.URL  \
0  https://

It turns out that many of these images don't actually include
that much data. 

In [13]:
results[4] # T48PXA

Collection: {'EntryTitle': 'HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': 106.93808729, 'Latitude': 13.47253979}, {'Longitude': 106.94640472, 'Latitude': 14.46477966}, {'Longitude': 105.92789032, 'Latitude': 14.47100048}, {'Longitude': 105.92392289, 'Latitude': 13.47831775}, {'Longitude': 106.93808729, 'Latitude': 13.47253979}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2023-11-21T03:34:10.537Z', 'EndingDateTime': '2023-11-21T03:34:10.537Z'}}
Size(MB): 210.91917896270752
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B12.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B08.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B07.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B04.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.Fmask.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.SZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B02.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B11.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B06.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B09.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B01.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.VAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B8A.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.SAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.VZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B05.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B03.tif']

# Choose an HLS image to use

I happen to know that the sentinel tile called 
48PXA includes all the sampling locations, so I will start 
with that. 

But I think I may have to end up doing this separately for 
each tile, unless there is some workaround via rioxarray for example. 

In [14]:
pxa_idx = results_df.loc[results_df['meta.native-id'].str.contains('T48PXA')].index
# turn it into a plain old list so I can use it to index results
pxa_results = [results[idx] for idx in pxa_idx]

Get urls for all the T48PXA images

In [15]:
hls_results_urls = [granule.data_links() for granule in pxa_results]
hls_results_urls[0:2] # Show a subset of the list

[['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T48PXA.2023325T031928.v2.0/HLS.L30.T48PXA.2023325T031928.v2.0.B03.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T48PXA.2023325T031928.v2.0/HLS.L30.T48PXA.2023325T031928.v2.0.Fmask.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T48PXA.2023325T031928.v2.0/HLS.L30.T48PXA.2023325T031928.v2.0.B10.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T48PXA.2023325T031928.v2.0/HLS.L30.T48PXA.2023325T031928.v2.0.SAA.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T48PXA.2023325T031928.v2.0/HLS.L30.T48PXA.2023325T031928.v2.0.B11.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T48PXA.2023325T031928.v2.0/HLS.L30.T48PXA.2023325T031928.v2.0.B07.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protect

Configure GDAL to successfully access HSL tiles

In [16]:

# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')

In [21]:
h = hls_results_urls[1]  
h

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B12.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B08.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B07.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B04.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.Fmask.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.SZA.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS

In [23]:
# Pull RGB
rgb_band_links = []

# Define which HLS product is being accessed
rgb_bands = ['B04', 'B03', 'B02'] # RGB

# Subset the assets in the item down to only the desired bands
for url in h: 
    if any(b in url for b in rgb_bands):
        rgb_band_links.append(url)
rgb_band_links

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B04.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B02.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B03.tif']

# Read HLS image into memory

HLS COGs are broken into chunks allowing data to be read more efficiently. Define the chunk size of an HLS tile, mask the NaN values, then read the files using rioxarray and name them based upon the band. We also squeeze the object to remove the band dimension from most of the files, since there is only 1 band.

In [30]:

# Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
chunk_size = dict(band=1, x=512, y=512) # Tiles have 1 band and are divided into 512x512 pixel chunks
# Sometimes a vsi curl error occurs so we need to retry if it does
max_retries = 10
for url in rgb_band_links:
    print(url)
    # Try Loop
    for _i in range(max_retries):
        try:
            # Open and build datasets
            if url.rsplit('.', 2)[-2] == rgb_bands[0]:      # red index
                nir = rxr.open_rasterio(url, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                nir.attrs['scale_factor'] = 0.0001        # hard coded the scale_factor attribute 
            elif url.rsplit('.', 2)[-2] == rgb_bands[1]:    # green index
                red = rxr.open_rasterio(url, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                red.attrs['scale_factor'] = 0.0001        # hard coded the scale_factor attribute
            elif url.rsplit('.', 2)[-2] == rgb_bands[2]:    # blue index
                blue = rxr.open_rasterio(url, chunks=chunk_size, masked=True).squeeze('band', drop=True)
                blue.attrs['scale_factor'] = 0.0001       # hard coded the scale_factor attribute
            break # Break out of the retry loop
        except Exception as ex:
            print(f"vsi curl error: {ex}. Retrying...")
    else:
        print(f"Failed to process {url} after {max_retries} retries. Please check to see you're authenticated with earthaccess.")
print("The COGs have been loaded into memory!")

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B04.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B02.tif
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T48PXA.2023325T032029.v2.0/HLS.S30.T48PXA.2023325T032029.v2.0.B03.tif
The COGs have been loaded into memory!


In [31]:
nir

<xarray.DataArray (y: 3660, x: 3660)>
dask.array<getitem, shape=(3660, 3660), dtype=float32, chunksize=(512, 512), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 6e+05 6e+05 6.001e+05 ... 7.098e+05 7.098e+05
  * y            (y) float64 1.6e+06 1.6e+06 1.6e+06 ... 1.49e+06 1.49e+06
    spatial_ref  int64 0
Attributes: (12/41)
    ACCODE:                                            LaSRC
    add_offset:                                        0.0
    AREA_OR_POINT:                                     Area
    arop_ave_xshift(meters):                           0
    arop_ave_yshift(meters):                           0
    arop_ncp:                                          0
    ...                                                ...
    SPACECRAFT_NAME:                                   Sentinel-2B
    spatial_coverage:                                  99
    SPATIAL_RESOLUTION:                                30
    TILE_ID:                                           S2B_OPER_MSI_L1C_TL_2B...
    ULX:                                               600000
    ULY:                                               1600020

Get the CRS for HLS tiles (It's UTM zone 48N)

In [33]:
fsUTM = sampling_points.to_crs(red.spatial_ref.crs_wkt) # Take the CRS from the NIR tile that we opened and apply it to our field geodataframe.
fsUTM

ele                      time      name  \
0   41.848339 2023-11-21 01:12:03+00:00  KSCS2-S1   
3   46.226585 2023-11-21 03:19:29+00:00     STS-1   
6   50.485073 2023-11-21 04:51:37+00:00     3SB-1   
9   44.720238 2023-11-21 08:39:23+00:00     SKG-1   
12  41.572842 2023-11-21 09:37:06+00:00     SSN-1   

                          geometry  
0   POINT (606101.431 1500960.051)  
3   POINT (602019.491 1495497.833)  
6   POINT (604961.861 1496949.311)  
9   POINT (613835.252 1499406.775)  
12  POINT (611893.715 1498311.861)